### All days of the challange:

* [Day 1: Handling missing values](https://www.kaggle.com/rtatman/data-cleaning-challenge-handling-missing-values)
* [Day 2: Scaling and normalization](https://www.kaggle.com/rtatman/data-cleaning-challenge-scale-and-normalize-data)
* [Day 3: Parsing dates](https://www.kaggle.com/rtatman/data-cleaning-challenge-parsing-dates/)
* [Day 4: Character encodings](https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings/)
* [Day 5: Inconsistent Data Entry](https://www.kaggle.com/rtatman/data-cleaning-challenge-inconsistent-data-entry/)
___
¡Bienvenido al día 4 del Desafío de datos de 5 días! Hoy trabajaremos con diferentes codificaciones de caracteres. Para comenzar, haga clic en el botón azul "Fork Notebook" en la esquina superior derecha. Esto creará una copia privada de este cuaderno que puede editar y jugar. Una vez que haya terminado con los ejercicios, puede optar por hacer público su cuaderno para compartirlo con otros. :)

> **¡Tu turno!** Mientras trabajamos en este cuaderno, verá algunas celdas del cuaderno (un bloque de código o texto) que tiene "¡Tu turno!" escrito en él. Estos son ejercicios que puede realizar para ayudar a consolidar su comprensión de los conceptos de los que estamos hablando. Una vez que haya escrito el código para responder una pregunta específica, puede ejecutar el código haciendo clic dentro de la celda (cuadro con el código) con el código que desea ejecutar y luego presione CTRL + ENTER (CMD + ENTER en una Mac ). También puede hacer clic en una celda y luego hacer clic en la flecha derecha "reproducir" a la izquierda del código. Si desea ejecutar todo el código en su cuaderno, puede usar las flechas dobles de "avance rápido" en la parte inferior del editor del cuaderno.

Esto es lo que haremos hoy:

* [Get our environment set up](#Get-our-environment-set-up)
* [What are encodings?](#What-are-encodings?)
* [Reading in files with encoding problems](#Reading-in-files-with-encoding-problems)
* [Saving your files with UTF-8 encoding](#Saving-your-files-with-UTF-8-encoding)

¡Empecemos!

# Configurar nuestro entorno
________

Lo primero que tendremos que hacer es cargar las bibliotecas que usaremos. Sin embargo, no nuestros conjuntos de datos: ¡llegaremos a ellos más tarde!

> **¡Importante!** ¡Asegúrese de ejecutar esta celda usted mismo o el resto de su código no funcionará!

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful character encoding module
import chardet

# set seed for reproducibility
np.random.seed(0)

¡Ahora estamos listos para trabajar con algunas codificaciones de caracteres! (Si lo desea, puede agregar una celda de código aquí y aprovechar esta oportunidad para ver algunos de los datos).

# ¿Qué son las codificaciones?
____

Las codificaciones de caracteres son conjuntos específicos de reglas para mapear desde cadenas de bytes binarios sin procesar (que se ven así: 0110100001101001) a caracteres que componen texto legible por humanos (como "hola"). Hay muchas codificaciones diferentes, y si trataste de leer un texto con una codificación diferente a la que estaba escrito originalmente, terminaste con un texto codificado llamado "mojibake" (dicho como mo-gee-bah-kay). Aquí tienes un ejemplo de mojibake:

æ– ‡ å — åŒ – ã ??

También puede terminar con personajes "desconocidos". Hay lo que se imprime cuando no hay un mapeo entre un byte en particular y un carácter en la codificación que está usando para leer su cadena de bytes y se ven así:

����������

Las discrepancias en la codificación de caracteres son menos comunes hoy en día de lo que solían ser, pero definitivamente sigue siendo un problema. Hay muchas codificaciones de caracteres diferentes, pero la principal que necesita saber es UTF-8.

> UTF-8 is **the** codificación de texto estándar. Todo el código Python está en UTF-8 e, idealmente, todos sus datos también deberían estarlo. Es cuando las cosas no están en UTF-8 cuando te metes en problemas.

Fue bastante difícil lidiar con codificaciones en Python 2, pero afortunadamente en Python 3 es mucho más simple. (Los kernels de Kaggle solo usan Python 3.) Hay dos tipos de datos principales que encontrará cuando trabaje con texto en Python 3. Uno es la cadena, que es el texto predeterminado.

In [ ]:
# start with a string
before = "This is the euro symbol: €"

# check to see what datatype it is
type(before)

The other data is the [bytes](https://docs.python.org/3.1/library/functions.html#bytes) tipo de datos, que es una secuencia de números enteros. Puede convertir una cadena en bytes especificando en qué codificación está:

In [ ]:
# encode it to a different encoding, replacing characters that raise errors
after = before.encode("utf-8", errors = "replace")

# check the type
type(after)

Si observa un objeto de bytes, verá que tiene una b delante de él, y luego tal vez algo de texto después. Eso es porque los bytes se imprimen como si fueran caracteres codificados en ASCII. (ASCII es una codificación de caracteres más antigua que realmente no funciona para escribir en ningún idioma que no sea el inglés). Aquí puede ver que nuestro símbolo del euro ha sido reemplazado por un mojibake que se parece a "\ xe2 \ x82 \ xac" cuando está impreso como si fuera una cadena ASCII.

In [ ]:
# take a look at what the bytes look like
after

Cuando volvemos a convertir nuestros bytes en una cadena con la codificación correcta, podemos ver que todo nuestro texto está allí correctamente, ¡lo cual es genial! :)

In [ ]:
# convert it back to utf-8
print(after.decode("utf-8"))

Sin embargo, cuando intentamos usar una codificación diferente para mapear nuestros bytes en una cadena, obtenemos un error. Esto se debe a que la codificación que estamos tratando de usar no sabe qué hacer con los bytes que estamos tratando de pasar. Debe decirle a Python la codificación en la que se supone que está realmente la cadena de bytes.

> Puede pensar en diferentes codificaciones como diferentes formas de grabar música. Puede grabar la misma música en un CD, cinta de casete o de 8 pistas. Si bien la música puede sonar más o menos igual, debe usar el equipo adecuado para reproducir la música de cada formato de grabación. El decodificador correcto es como un reproductor de casetes o un reproductor de CD. Si intenta reproducir un casete en un reproductor de CD, simplemente no funcionará.

In [ ]:
# try to decode our bytes with the ascii encoding
print(after.decode("ascii"))

También podemos tener problemas si intentamos usar la codificación incorrecta para mapear de una cadena a bytes. Como dije antes, las cadenas son UTF-8 por defecto en Python 3, así que si tratamos de tratarlas como si estuvieran en otra codificación, crearemos problemas.

Por ejemplo, si intentamos convertir una cadena en bytes para ascii usando encode (), podemos pedir que los bytes sean los que serían si el texto estuviera en ASCII. Sin embargo, dado que nuestro texto no está en ASCII, habrá algunos caracteres que no podrá manejar. Podemos reemplazar automáticamente los caracteres que ASCII no puede manejar. Sin embargo, si hacemos eso, cualquier carácter que no esté en ASCII será reemplazado por el carácter desconocido. Luego, cuando volvamos a convertir los bytes en una cadena, el carácter será reemplazado por el carácter desconocido. La parte peligrosa de esto es que no hay forma de saber qué personaje * debería * haber sido. ¡Eso significa que es posible que hayamos inutilizado nuestros datos!

In [ ]:
# start with a string
before = "This is the euro symbol: €"

# encode it to a different encoding, replacing characters that raise errors
after = before.encode("ascii", errors = "replace")

# convert it back to utf-8
print(after.decode("ascii"))

# We've lost the original underlying byte string! It's been 
# replaced with the underlying byte string for the unknown character :(

¡Esto es malo y queremos evitarlo! Es mucho mejor convertir todo nuestro texto a UTF-8 tan pronto como podamos y mantenerlo en esa codificación. El mejor momento para convertir una entrada que no sea UTF-8 en UTF-8 es cuando lee archivos, de lo que hablaremos a continuación.

Primero, sin embargo, intente convertir entre bytes y cadenas con diferentes codificaciones y vea qué sucede. Observe lo que esto le hace a su texto. ¿Le gustaría que esto sucediera con los datos que estaba tratando de analizar?

In [ ]:
# Your turn! Try encoding and decoding different symbols to ASCII and
# see what happens. I'd recommend $, #, 你好 and नमस्ते but feel free to
# try other characters. What happens? When would this cause problems?

# Lectura de archivos con problemas de codificación
___

La mayoría de los archivos que encontrará probablemente estarán codificados con UTF-8. Esto es lo que Python espera de forma predeterminada, por lo que la mayoría de las veces no tendrá problemas. Sin embargo, a veces obtendrá un error como este:

In [ ]:
# try to read in a file not in UTF-8
kickstarter_2016 = pd.read_csv("../input/kickstarter-projects/ks-projects-201612.csv")

¡Observe que obtenemos el mismo `UnicodeDecodeError` que obtuvimos cuando intentamos decodificar bytes UTF-8 como si fueran ASCII! Esto nos dice que este archivo no es en realidad UTF-8. Sin embargo, no sabemos qué codificación * es *. Una forma de averiguarlo es probar y probar un montón de codificaciones de caracteres diferentes y ver si alguna de ellas funciona. Sin embargo, una mejor manera es usar el módulo chardet para intentar adivinar automáticamente cuál es la codificación correcta. No se garantiza al 100% que sea correcto, pero generalmente es más rápido que solo intentar adivinar.

Solo voy a mirar los primeros diez mil bytes de este archivo. Esto suele ser suficiente para adivinar cuál es la codificación y es mucho más rápido que intentar ver el archivo completo. (Especialmente con un archivo grande, esto puede ser muy lento). Otra razón para mirar la primera parte del archivo es que podemos ver al mirar el mensaje de error que el primer problema es el undécimo carácter. Así que probablemente solo necesitemos mirar el primer fragmento del archivo para averiguar qué está pasando.

In [ ]:
# look at the first ten thousand bytes to guess the character encoding
with open("../input/kickstarter-projects/ks-projects-201801.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)

So chardet is 73%  confidence that the right encoding is "Windows-1252". Let's see if that's correct:

In [ ]:
# read in the file with the encoding detected by chardet
kickstarter_2016 = pd.read_csv("../input/kickstarter-projects/ks-projects-201612.csv", encoding='Windows-1252')

# look at the first few lines
kickstarter_2016.head()

¡Sí, parece que Chardet tenía razón! El archivo se lee sin problemas (aunque recibimos una advertencia sobre los tipos de datos) y cuando miramos las primeras filas parece estar bien. 

> **¿Qué pasa si la codificación de Chardet adivina no es correcta?** Dado que chardet es básicamente un adivino sofisticado, a veces adivinará la codificación incorrecta. Una cosa que puede intentar es mirar más o menos el archivo y ver si obtiene un resultado diferente y luego intentarlo.

In [ ]:
# Your Turn! Trying to read in this file gives you an error. Figure out
# what the correct encoding should be and read in the file. :)
police_killings = pd.read_csv("../input/fatal-police-shootings-in-the-us/PoliceKillingsUS.csv")

# Guardar sus archivos con codificación UTF-8
___

Finalmente, una vez que haya pasado por todos los problemas de llevar su archivo a UTF-8, probablemente querrá mantenerlo así. La forma más sencilla de hacerlo es guardar sus archivos con codificación UTF-8. La buena noticia es que, dado que UTF-8 es la codificación estándar en Python, cuando guarde un archivo, se guardará como UTF-8 de forma predeterminada:

In [ ]:
# save our file (will be saved as UTF-8 by default!)
kickstarter_2016.to_csv("ks-projects-201801-utf8.csv")

Bastante fácil, ¿eh? :)

> Si no ha guardado un archivo en un kernel antes, debe presionar el botón confirmar y ejecutar y esperar a que su cuaderno termine de ejecutarse antes de poder ver o acceder al archivo que ha guardado. Si no lo ve al principio, espere un par de minutos y debería aparecer. Los archivos que guarde estarán en el directorio "../output/", y puede descargarlos desde su computadora portátil.

In [ ]:
# Your turn! Save out a version of the police_killings dataset with UTF-8 encoding 

¡Y eso es todo por hoy! No hicimos tanta codificación, pero confíe en mi palabra: si no tiene las herramientas adecuadas, averiguar en qué codificación se encuentra un archivo puede suponer una gran pérdida de tiempo. Si tiene alguna pregunta, asegúrese de publicarla en los comentarios a continuación o [on the forums](https://www.kaggle.com/questions-and-answers). 

Recuerde que su bloc de notas es privado de forma predeterminada y, para compartirlo con otras personas o pedir ayuda con él, deberá hacerlo público. Primero, deberá guardar una versión de su cuaderno que muestre su trabajo actual presionando el botón "Confirmar y ejecutar". (Su trabajo se guarda automáticamente, pero el control de versiones de su trabajo le permite volver atrás y ver cómo era en el momento en que lo guardó. También le permite compartir un buen cuaderno compilado en lugar de solo el código sin procesar). la computadora portátil ha terminado de ejecutarse, puede ir a la pestaña Configuración en el panel de la izquierda (es posible que deba expandirla presionando el botón [<] junto al botón "Confirmar y ejecutar") y configurar el menú desplegable "Visibilidad" en "Público".

# ¡Más práctica!
___

Verificar [this dataset of files in different character encodings](https://www.kaggle.com/rtatman/character-encoding-examples). ¿Puede leer todos los archivos con sus codificaciones originales y guardarlos como archivos UTF-8?

Si tiene un archivo en UTF-8 pero tiene solo un par de caracteres extraños, puede probar el [ftfy module](https://ftfy.readthedocs.io/en/latest/#) y vea si ayuda.